In [ ]:
# Install mlcroissant from the source
!apt-get install -y python3-dev graphviz libgraphviz-dev pkg-config
!pip install "git+https://github.com/${GITHUB_REPOSITORY:-mlcommons/croissant}.git@${GITHUB_HEAD_REF:-main}#subdirectory=python/mlcroissant&egg=mlcroissant[dev]"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
graphviz is already the newest version (2.42.2-6ubuntu0.1).
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libgvc6-plugins-gtk
  librsvg2-common libxdot4
Suggested packages:
  gvfs
The following packages will be REMOVED:
  pkgconf r-base-dev
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk librsvg2-common libxdot4 pkg-config
0 upgraded, 10 newly installed, 2 to remove and 49 not upgraded.
Need to get 2,482 kB of archives.
After this operation, 7,670 kB of add

Virginia Food Banks Dataset

In [ ]:
import mlcroissant as mlc
import pandas as pd

# Fetch the Croissant JSON-LD
croissant_dataset = mlc.Dataset('www.kaggle.com/datasets/dsrajpatel/virginia-food-banks/croissant/download')

# Check what record sets are in the dataset
record_sets = croissant_dataset.metadata.record_sets
print(record_sets)

# Fetch the records and put them in a DataFrame
record_set_df = pd.DataFrame(croissant_dataset.records(record_set=record_sets[0].uuid))
record_set_df.head()


/usr/local/lib/python3.10/dist-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


ValueError: File /content/www.kaggle.com/datasets/dsrajpatel/virginia-food-banks/croissant/download does not exist.

In [66]:
import pandas as pd

def loadVAFoodBackData():
  VA_fb = pd.read_csv("VA_food_bank.csv")
  return VA_fb

VA_fb = loadVAFoodBackData()
VA_fb.info()
VA_fb.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4259 entries, 0 to 4258
Data columns (total 13 columns):
 #   Column                                                            Non-Null Count  Dtype  
---  ------                                                            --------------  -----  
 0   Year                                                              4259 non-null   int64  
 1   Month                                                             4259 non-null   object 
 2   FIPS                                                              4256 non-null   float64
 3   Locality                                                          4259 non-null   object 
 4   Households Served                                                 4029 non-null   float64
 5   Individuals Served                                                4029 non-null   float64
 6   Pounds of Food Distributed                                        4029 non-null   float64
 7   Children Served via non-federal c

,Year,Month,FIPS,Locality,Households Served,Individuals Served,Pounds of Food Distributed,Children Served via non-federal child nutrition programs,Pounds of food distributed via non-federal child nutrition progr,LAT,LON,MTW Status,Geopoint
0,2020,October,97.0,King and Queen,NaN,NaN,156644.0,NaN,NaN,37.714370,-76.901863,Non-matching,POINT (-76.901863 37.7143698)
1,2019,April,115.0,Mathews County,NaN,NaN,21602.0,0.0,NaN,37.425348,-76.268808,Non-matching,POINT (-76.2688078 37.4253479)
2,2019,August,97.0,King and Queen,NaN,NaN,51338.0,NaN,NaN,37.714370,-76.901863,Non-matching,POINT (-76.901863 37.7143698)


In [67]:
missing = VA_fb.isnull().sum()
print(missing)

Year                                                                   0
Month                                                                  0
FIPS                                                                   3
Locality                                                               0
Households Served                                                    230
Individuals Served                                                   230
Pounds of Food Distributed                                           230
Children Served via non-federal child nutrition programs            2983
Pounds of food distributed via non-federal child nutrition progr    1567
LAT                                                                    0
LON                                                                    0
MTW Status                                                             0
Geopoint                                                               0
dtype: int64


In [68]:
#Removing data that may not be needed, can just remove a columns name from the list below to add it back
VA_fb = VA_fb.drop(columns=['Children Served via non-federal child nutrition programs','LAT','LON','MTW Status','Geopoint','Pounds of food distributed via non-federal child nutrition progr'])
VA_fb.head(3)

,Year,Month,FIPS,Locality,Households Served,Individuals Served,Pounds of Food Distributed
0,2020,October,97.0,King and Queen,NaN,NaN,156644.0
1,2019,April,115.0,Mathews County,NaN,NaN,21602.0
2,2019,August,97.0,King and Queen,NaN,NaN,51338.0


In [71]:
from sklearn.impute import SimpleImputer

missing = VA_fb.isnull().sum()
print(missing)

imputer = SimpleImputer(strategy='mean')
numeric_columns = VA_fb.select_dtypes(include=['number']).columns
VA_fb[numeric_columns] = imputer.fit_transform(VA_fb[numeric_columns])
print('-------------------------------')
missing = VA_fb.isnull().sum()
print(missing)

VA_fb.head(3)

Year                          0
Month                         0
FIPS                          0
Locality                      0
Households Served             0
Individuals Served            0
Pounds of Food Distributed    0
dtype: int64
-------------------------------
Year                          0
Month                         0
FIPS                          0
Locality                      0
Households Served             0
Individuals Served            0
Pounds of Food Distributed    0
dtype: int64


,Year,Month,FIPS,Locality,Households Served,Individuals Served,Pounds of Food Distributed
0,2020.0,October,97.0,King and Queen,908.984363,2971.508771,156644.0
1,2019.0,April,115.0,Mathews County,908.984363,2971.508771,21602.0
2,2019.0,August,97.0,King and Queen,908.984363,2971.508771,51338.0


In [39]:
def loadPoverty():
  poverty = pd.read_csv('rows.csv')
  return poverty

poverty = loadPoverty()
poverty.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799 entries, 0 to 798
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   year         799 non-null    object
 1   county       799 non-null    object
 2   B17020_001E  799 non-null    int64 
 3   B17020_002E  799 non-null    int64 
 4   B17020_003E  799 non-null    int64 
 5   B17020_004E  799 non-null    int64 
 6   B17020_005E  799 non-null    int64 
 7   state        799 non-null    int64 
 8   county 1     799 non-null    int64 
dtypes: int64(7), object(2)
memory usage: 56.3+ KB


In [40]:
missing = poverty.isnull().sum()
missing

,0
year,0
county,0
B17020_001E,0
B17020_002E,0
B17020_003E,0
B17020_004E,0
B17020_005E,0
state,0
county 1,0


In [41]:
poverty.head(3)

,year,county,B17020_001E,B17020_002E,B17020_003E,B17020_004E,B17020_005E,state,county 1
0,01/01/2018 12:00:00 AM,"Tazewell County, Virginia",40559,6550,895,711,351,51,185
1,01/01/2015 12:00:00 AM,"Accomack County, Virginia",32582,6384,741,828,497,51,1
2,01/01/2015 12:00:00 AM,"Albemarle County, Virginia",95348,9714,735,502,829,51,3


In [42]:
#Dropping state since ours is just virginia
poverty = poverty.drop(columns=['state'])
#Combining all the poverty numbers into one column
columns_to_sum = ['B17020_001E', 'B17020_002E', 'B17020_003E', 'B17020_004E', 'B17020_005E']
poverty['people_below_poverty_total'] = poverty[columns_to_sum].sum(axis=1)
poverty = poverty.drop(columns=columns_to_sum)
#Changing date to only contain the year
poverty['year'] = pd.to_datetime(poverty['year']).dt.year
#Removing ",Virginia" from each county name so it can be matched with the food_bank if needed
poverty['county'] = poverty['county'].str.replace(', Virginia', '', regex=False)

poverty.head(3)

<ipython-input-42-ace5110d3947>:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  poverty['year'] = pd.to_datetime(poverty['year']).dt.year


,year,county,county 1,people_below_poverty_total
0,2018,Tazewell County,185,49066
1,2015,Accomack County,1,41032
2,2015,Albemarle County,3,107128
